In [1]:
# TODO: Set up the environments and check if they work out
# TODO: Create DQN algorithm as a class
# TODO: Train it for epsilon=0.1, step_size=0.001, 1 seed for 1000 episodes, save the performance and the model
# TODO: 
# TODO: 
# TODO: Create Expected Sarsa (look up previous assignment)
# TODO: 

In [69]:
!pip install gymnasium
!pip install ale-py
!pip install gymnasium[classic-control]
!pip install torch

In [1]:
import gymnasium as gym
import ale_py
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import matplotlib.pyplot as plt
from collections import deque

In [2]:
gym.register_envs(ale_py)

envs = {
    "acrobot": gym.make('Acrobot-v1', render_mode="rgb_array"),
    "assault": gym.make('ALE/Assault-ram-v5', render_mode="rgb_array")
}

In [3]:
envs["acrobot"]

In [4]:
envs["acrobot"].reset(seed=123, options={"low": -0.2, "high": 0.2})

In [5]:
for env_name,env in envs.items():
    print(f"Environment: {env_name}")
    print(f"State Space: {env.observation_space}")
    # print(f"State Space Type: {type(env.observation_space)}")
    print("Action space:", env.action_space)
    # print(f"Action Space Type: {type(env.action_space)}")

In [6]:
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import matplotlib.pyplot as plt
from collections import deque

# Neural Network (MLP) for Function Approximation
class MLP(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_dim)
        
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.uniform_(m.weight, -0.01, 0.01)
                nn.init.constant_(m.bias, 0.01)  #  initialize biases to 0.1

    def forward(self, x):
            x = torch.relu(self.fc1(x))
            x = torch.relu(self.fc2(x))
            return self.fc3(x)

# Experience Replay Buffer
class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def push(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        return (
            torch.tensor(states, dtype=torch.float),
            torch.tensor(actions, dtype=torch.int64),
            torch.tensor(rewards, dtype=torch.float),
            torch.tensor(next_states, dtype=torch.float),
            torch.tensor(dones, dtype=torch.bool),
        )

    def __len__(self):
        return len(self.buffer)

# DQN Agent
class DQN:
    def __init__(self, state_dim, action_dim, replay_buffer=True, learning_rate=1e-3, gamma=0.99, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.995):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.replay_buffer = replay_buffer  # True -> Uses Replay Buffer, False -> Learns from latest transition
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.batch_size = 32
        self.target_update_freq = 10

        # Q-network and target network
        self.q_network = MLP(state_dim, action_dim)
        self.target_network = MLP(state_dim, action_dim)
        self.target_network.load_state_dict(self.q_network.state_dict())  # Copy initial weights

        self.optimizer = optim.Adam(self.q_network.parameters(), lr=learning_rate)
        self.loss_fn = nn.MSELoss()

        # Initialize replay buffer if needed
        self.buffer = ReplayBuffer(capacity=10000) if self.replay_buffer else None

        # If no replay buffer, store the last transition
        self.last_transition = None

    def select_action(self, state):
        """Epsilon-greedy action selection."""
        if random.random() < self.epsilon:
            return random.randint(0, self.action_dim - 1)
        else:
            state = torch.tensor(state, dtype=torch.float).unsqueeze(0)
            with torch.no_grad():
                return torch.argmax(self.q_network(state)).item()

    def train(self):
        """Train the agent using a batch of experiences from replay buffer."""
        if self.replay_buffer:
            if len(self.buffer) < self.batch_size:
                return  # Wait until we have enough data
    
            # Sample a batch from replay buffer
            states, actions, rewards, next_states, dones = self.buffer.sample(self.batch_size)
        else:
            if self.last_transition is None:
                return
    
            # Convert single transition to a batch of size 1
            states, actions, rewards, next_states, dones = self.last_transition
            states, actions, rewards, next_states, dones = (
                states.unsqueeze(0),
                actions.unsqueeze(0),
                rewards.unsqueeze(0),
                next_states.unsqueeze(0),
                dones.unsqueeze(0),
            )
    
        # Compute Q-values for selected actions
        q_values = self.q_network(states).gather(1, actions.unsqueeze(1)).squeeze(1)
    
        # Compute target Q-values
        with torch.no_grad():
            next_q_values = self.target_network(next_states).max(1)[0]
            target_q_values = rewards + self.gamma * next_q_values * (~dones)
    
        # Compute loss (Mean Squared Error)
        loss = self.loss_fn(q_values, target_q_values)
    
        # Optimize the network
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()


    def update_target_network(self):
        """Copy weights from Q-network to target network."""
        self.target_network.load_state_dict(self.q_network.state_dict())

    def decay_epsilon(self):
        """Decay exploration rate."""
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)


In [13]:
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import matplotlib.pyplot as plt
from collections import deque

# Neural Network (MLP) for Function Approximation
class MLP(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_dim)
        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.uniform_(m.weight, -0.01, 0.01)  # Uniform distribution [-0.01, 0.01]
                nn.init.constant_(m.bias, 0.01)  # Initialize biases to 0.01

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# Experience Replay Buffer
class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def push(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        return (
            torch.tensor(states, dtype=torch.float),
            torch.tensor(actions, dtype=torch.int64),
            torch.tensor(rewards, dtype=torch.float),
            torch.tensor(next_states, dtype=torch.float),
            torch.tensor(dones, dtype=torch.bool),
        )

    def __len__(self):
        return len(self.buffer)

# DQN Agent
class DQN:
    def __init__(self, state_dim, action_dim, replay_buffer=True, learning_rate=1e-3, gamma=0.99,
                 epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.995):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.replay_buffer = replay_buffer  # True -> Uses Replay Buffer, False -> Learns from latest transition
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.batch_size = 64
        self.target_update_freq = 10

        # Q-network and target network
        self.q_network = MLP(state_dim, action_dim)
        self.target_network = MLP(state_dim, action_dim)
        self.target_network.load_state_dict(self.q_network.state_dict())  # Copy initial weights

        self.optimizer = optim.Adam(self.q_network.parameters(), lr=learning_rate)
        self.loss_fn = nn.MSELoss()

        # Initialize replay buffer if needed
        self.buffer = ReplayBuffer(capacity=10000) if self.replay_buffer else None

        # If no replay buffer, store the last transition
        self.last_transition = None

    def select_action(self, state):
        """Epsilon-greedy action selection."""
        if random.random() < self.epsilon:
            return random.randint(0, self.action_dim - 1)
        else:
            state = torch.tensor(state, dtype=torch.float).unsqueeze(0)
            with torch.no_grad():
                return torch.argmax(self.q_network(state)).item()

    def store_transition(self, state, action, reward, next_state, done):
        """Stores transition in replay buffer or keeps the latest one."""
        if self.replay_buffer:
            self.buffer.push(state, action, reward, next_state, done)
        else:
            self.last_transition = (
                torch.tensor([state], dtype=torch.float),
                torch.tensor([action], dtype=torch.int64),
                torch.tensor([reward], dtype=torch.float),
                torch.tensor([next_state], dtype=torch.float),
                torch.tensor([done], dtype=torch.bool),
            )

    def train(self):
        """Train the agent using replay buffer or latest transition."""
        if self.replay_buffer:
            # Ensure there is enough data in the buffer
            if len(self.buffer) < self.batch_size:
                return

            # Sample batch from replay buffer
            states, actions, rewards, next_states, dones = self.buffer.sample(self.batch_size)
        else:
            # Train only on the last transition (No replay buffer)
            if self.last_transition is None:
                return
            states, actions, rewards, next_states, dones = self.last_transition

        # Compute Q-values
        q_values = self.q_network(states).gather(1, actions.unsqueeze(1)).squeeze(1)
        next_q_values = self.target_network(next_states).max(1)[0].detach()
        target_q_values = rewards + self.gamma * next_q_values * (~dones)

        # Compute loss
        loss = self.loss_fn(q_values, target_q_values)

        # Optimize model
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def update_target_network(self):
        """Copy weights from Q-network to target network."""
        self.target_network.load_state_dict(self.q_network.state_dict())

    def decay_epsilon(self):
        """Decay exploration rate."""
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)


In [7]:
env = envs["acrobot"]

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

# Set replay_buffer=True to use replay buffer, False to disable it
agent = DQN(state_dim, action_dim, replay_buffer=True)

num_episodes = 1000
reward_history = []

for episode in range(num_episodes):
    state, _ = env.reset()
    total_reward = 0

    for t in range(500):  # Max steps per episode 500 for acrobot, 1000 for assult
        action = agent.select_action(state)
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

        if agent.replay_buffer:
            agent.buffer.push(state, action, reward, next_state, done)
        else:
            # Store the last transition in a single tuple
            agent.last_transition = (
                torch.tensor([state], dtype=torch.float),
                torch.tensor([action], dtype=torch.int64),
                torch.tensor([reward], dtype=torch.float),
                torch.tensor([next_state], dtype=torch.float),
                torch.tensor([done], dtype=torch.bool),
            )

        state = next_state
        total_reward += reward

        # Train the agent
        agent.train()

        if done:
            break

    # Update target network every few episodes
    if episode % agent.target_update_freq == 0:
        agent.update_target_network()

    # Decay epsilon
    agent.decay_epsilon()

    reward_history.append(total_reward)
    print(f"Episode {episode}, Reward: {total_reward}, Epsilon: {agent.epsilon:.3f}")

env.close()


In [8]:
plt.plot(reward_history)
plt.xlabel("Episode")
plt.ylabel("Total Reward")
plt.title("DQN Training Performance")
plt.show()


In [11]:
env = gym.make("Acrobot-v1", render_mode="human")

state, _ = env.reset()
for _ in range(500):
    action = agent.select_action(state)
    state, _, terminated, truncated, _ = env.step(action)
    if terminated or truncated:
        break

env.close()


In [16]:
env = envs["acrobot"]

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

# Set replay_buffer=True to use replay buffer, False to disable it
agent = DQN(state_dim, action_dim, replay_buffer=False)

num_episodes = 1000
reward_history_no_replay = []

for episode in range(num_episodes):
    state, _ = env.reset()
    total_reward = 0

    for t in range(500):  # Max steps per episode 500 for acrobot, 1000 for assult
        action = agent.select_action(state)
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

        if agent.replay_buffer:
            agent.buffer.push(state, action, reward, next_state, done)
        else:
            # Store the last transition in a single tuple
            agent.last_transition = (
                torch.tensor([state], dtype=torch.float),
                torch.tensor([action], dtype=torch.int64),
                torch.tensor([reward], dtype=torch.float),
                torch.tensor([next_state], dtype=torch.float),
                torch.tensor([done], dtype=torch.bool),
            )

        state = next_state
        total_reward += reward

        # Train the agent
        agent.train()

        if done:
            break

    # Update target network every few episodes
    if episode % agent.target_update_freq == 0:
        agent.update_target_network()

    # Decay epsilon
    agent.decay_epsilon()

    reward_history_no_replay.append(total_reward)
    print(f"Episode {episode}, Reward: {total_reward}, Epsilon: {agent.epsilon:.3f}")

env.close()


In [17]:
plt.plot(reward_history_no_replay)
plt.xlabel("Episode")
plt.ylabel("Total Reward")
plt.title("DQN Training Performance")
plt.show()


In [18]:
env = gym.make("Acrobot-v1", render_mode="human")

state, _ = env.reset()
for _ in range(500):
    action = agent.select_action(state)
    state, _, terminated, truncated, _ = env.step(action)
    if terminated or truncated:
        break

env.close()
